# Importing the libraries

## Function to install/Uninstall libraries

In [ ]:
import sys
import subprocess
assert sys.version_info >= (3,5)

def install_packages(pkg_name, 
                     pkg_version = '', 
                     medium = 'pip'):
    
    print(f'Installing {pkg_name} {pkg_version}...',end='')
    try:
        if len(pkg_version) == 0:
            subprocess.check_call([sys.executable,
                                   '-m',
                                   medium,
                                   'install',
                                   pkg_name])
#             !{sys.executable} -m pip install --yes <pkg_name>
#             !conda install --yes --predix {sys.prefix} <pkg_name>
        else:
            subprocess.check_call([sys.executable,
                                   '-m',
                                   medium,
                                   'install',
                                   pkg_name+'=='+pkg_version])
        print('Installed successfully.')
    except:
        print(f'Unable to install! Please install it manually.')
    
def uninstall_packages(pkg_name, 
                     medium = 'pip'):
    
    print(f'Uninstalling {pkg_name}...',end='')
    try:
        subprocess.check_call([sys.executable,
                               '-m',
                               medium,
                               'uninstall',
                               pkg_name,
                               '-y'])
#             !{sys.executable} -m pip uninstall --yes <pkg_name>
#             !conda uninstall --yes --predix {sys.prefix} <pkg_name>
        print('Uninstalled successfully.')
    except:
        print(f'Unable to uninstall! Please uninstall it manually.')

## Installing openpyxl

In [ ]:
try:
    pkg_name = 'openpyxl'
    pkg_version = ''
    import openpyxl
    assert openpyxl.__version__ >= pkg_version
except ModuleNotFoundError:
    install_packages(pkg_name=pkg_name)
    import openpyxl
    assert openpyxl.__version__ >= pkg_version
except:
    print(f'Please install {pkg_name} {pkg_version} manually.')

## Installing pymatreader

In [ ]:
try:
    pkg_name = 'pymatreader'
    import pymatreader
except ModuleNotFoundError:
    install_packages(pkg_name=pkg_name)
    import pymatreader
except:
    print(f'Please install {pkg_name} {pkg_version} manually.')

## Installing Pands

In [ ]:
try:
    pkg_name = 'pandas'
    pkg_version = ''
    import pandas as pd
    assert pd.__version__ >= pkg_version
except ModuleNotFoundError:
    install_packages(pkg_name=pkg_name)
    import pandas as pd
    assert pd.__version__ >= pkg_version
except:
    print(f'Please install {pkg_name} {pkg_version} manually.')

## Importing Libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import pymatreader
import openpyxl

#  Class to load mat file using pymatreader and save to excel/database

In [ ]:
class mat_reader():
    
    def __init__(self, parent_dir):
        self.intialization(parent_dir)
        
    def intialization(self, parent_dir):
        self.count = 0
        self.parent_dir = parent_dir
        
    def _save_excel(self,
                    array, 
                    file_path):
        pd.DataFrame(array).to_excel(os.path.join(self.parent_dir, file_path), 
                              sheet_name=os.path.splitext(file_path)[0],
                              header=False,
                              index=False)
        
    def convert_mat_to_excel(self,
                             d,
                             indent=0,
                             nkeys=0,
                             file_name=''):
        if nkeys>0:
            d = {k:d[k] for k in list(d.keys())[:nkeys]}
            
        if isinstance(d,dict):
            wb = openpyxl.Workbook()
            ws1 = wb.active
            ws1.title = os.path.splitext(file_name)[0]
            row_no = 0
            for key,value in d.items():
                print('\t' * indent + f'Key: {key}')
                row_no += 1
                ws1.cell(row=row_no, column=1).value = f'{key}'
                if isinstance(value, dict):
                    self.count += 1
                    ws1.cell(row=row_no, column=2).hyperlink = f'{key}_{self.count}.xlsx'
                    ws1.cell(row=row_no, column=2).style = 'Hyperlink'
                    if isinstance(value[list(value.keys())[0]], list):
                        ws1.cell(row=row_no, column=2).value = f'({len(value)}x{len(value[list(value.keys())[0]])}) list'
#                         df = pd.DataFrame({ key:pd.Series(value) for key, value in value.items() })
                        df = pd.DataFrame(value, columns = list(value.keys()))
                        df.to_excel(excel_writer=os.path.join(self.parent_dir,f'{key}_{self.count}.xlsx'),
                                    sheet_name=f'{key}_{self.count}',
                                    header=True,
                                    index=False)
                        break
                    else:
                        ws1.cell(row=row_no, column=2).value = f'({len(value)}) dict'
                        self.convert_mat_to_excel(value, indent+1, file_name = f'{key}_{self.count}.xlsx')
                
                elif isinstance(value, np.ndarray):
                    self.count += 1
                    ws1.cell(row=row_no, column=2).value = f'({value.shape}) ndarray'
                    ws1.cell(row=row_no, column=2).hyperlink = f'{key}_{self.count}.xlsx'
                    ws1.cell(row=row_no, column=2).style = 'Hyperlink'
                    if value.dtype.names is not None:
                        self.convert_mat_to_excel(value, indent+1, file_name = f'{key}_{self.count}.xlsx')
                    else:
                        self._save_excel(value, f'{key}_{self.count}.xlsx')
                elif isinstance(value, list):
                    self.count += 1
                    ws1.cell(row=row_no, column=2).value = f'({len(value)}) list'
                    ws1.cell(row=row_no, column=2).hyperlink = f'{key}_{self.count}.xlsx'
                    ws1.cell(row=row_no, column=2).style = 'Hyperlink'
#                         df = pd.DataFrame({ key:pd.Series(value) for key, value in value.items() })
                    df = pd.DataFrame(value)
                    df.to_excel(excel_writer=os.path.join(self.parent_dir,f'{key}_{self.count}.xlsx'),
                                    sheet_name=f'{key}_{self.count}',
                                    header=True,
                                    index=False)
                else:
                    ws1.cell(row=row_no, column=2).value = f'{value}'
            wb.save(filename = os.path.join(self.parent_dir, file_name))
        elif isinstance(d,np.ndarray) and d.dtype.names is not None:  # Note: and short-circuits by default
            for n in d.dtype.names:    # This means it's a struct, it's bit of a kludge test.
                print('\t' * indent + 'Field: ' + str(n))
                self.count += 1
                self.convert_mat_to_excel(value, indent+1, file_name = f'{key}_{self.count}.xlsx')
       
    def loadmat(self, filename):
        return pymatreader.read_mat(filename)
                    

## Loading and converting mat file into excel

In [ ]:
def convert():
    mat_file_path = r'sample.mat'
    parent_dir = os.path.splitext(mat_file_path)[0]
    os.makedirs(parent_dir, exist_ok = True)
    m_reader  = mat_reader(parent_dir)
    m_reader.convert_mat_to_excel(m_reader.loadmat(mat_file_path), file_name = f'{mat_file_path}.xlsx')
convert()